In [1]:
%autosave 20
import math 
import pandas as pd

#import numpy as np
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap
%matplotlib inline
from sklearn import tree

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

Autosaving every 20 seconds


In [2]:
df=pd.read_csv("Sample dataset values.txt")
df.head(3)
df.shape
total_row=df.shape[0]
trial_row=int(total_row-(total_row*0.25))
print("total rows")

,Sr No,Sensor,Name,Variable,Units,Timestamp,Value,Flagged as Suspect Reading
0,1586958692652188245,sensor03,PER_AIRMON_MESH1976150,PM10,ugm -3,2018-12-13 03:16:00,16.12200,False
1,1586958692652348737,sensor01,PER_AIRMON_MESH1976150,PM 4,ugm -3,2018-12-14 12:46:00,5.19000,False
2,1586958692652681524,sensor02,PER_AIRMON_MESH1976150,NO,ugm -3,2018-12-14 00:30:00,0.08875,False


(267, 8)

total rows


In [3]:
def uni(col):
    print("There are ",len(df[col].unique())," unique elements  of column ", col," and they are ", df[col].unique())

In [4]:
list(map(uni,df[['Sensor', 'Name', 'Variable', 'Units','Flagged as Suspect Reading']]))

There are  3  unique elements  of column  Sensor  and they are  ['sensor03' 'sensor01' 'sensor02']
There are  1  unique elements  of column  Name  and they are  ['PER_AIRMON_MESH1976150']
There are  8  unique elements  of column  Variable  and they are  ['PM10' 'PM 4' 'NO' 'PM2.5' 'PM1' 'Particle Count' 'O3' 'NO2']
There are  3  unique elements  of column  Units  and they are  ['ugm -3' 'Kgm -3' 'ppb']
There are  2  unique elements  of column  Flagged as Suspect Reading  and they are  [False  True]


[None, None, None, None, None]

In [5]:
df.loc[df['Sensor']=='sensor01','Sensor']=0
df.loc[df['Sensor']=='sensor02','Sensor']=1
df.loc[df['Sensor']=='sensor03','Sensor']=2

In [6]:
df.loc[df['Variable']=='PM10','Variable']=0
df.loc[df['Variable']=='PM 4','Variable']=1
df.loc[df['Variable']=='NO','Variable']=2
df.loc[df['Variable']=='PM2.5','Variable']=3
df.loc[df['Variable']=='PM1','Variable']=4
df.loc[df['Variable']=='Particle Count','Variable']=5
df.loc[df['Variable']=='O3','Variable']=6
df.loc[df['Variable']=='NO2','Variable']=7

In [7]:
df.loc[df['Name']=='PER_AIRMON_MESH1976150','Name']=0

In [8]:
df.loc[df['Units']=='ugm -3','Units']=0
df.loc[df['Units']=='Kgm -3','Units']=1
df.loc[df['Units']=='ppb','Units']=2

In [9]:
df.loc[df['Flagged as Suspect Reading']==False,'Flagged as Suspect Reading']=0
df.loc[df['Flagged as Suspect Reading']==True,'Flagged as Suspect Reading']=1

In [10]:
x=['Sensor', 'Name', 'Variable', 'Units','Value']
y=['Flagged as Suspect Reading']
train=df.loc[1:trial_row,]
test=df.loc[trial_row+1:total_row,]

In [11]:
# train[['Value']]=train[['Value']].astype(int)
# test[['Value']]=test[['Value']].astype(int)

In [12]:
x_train,y_train,x_test,y_test = train[x],train[y],test[x],test[y]

In [13]:
x_train.head()
y_train.head()
x_test.head()
y_test.head()

,Sensor,Name,Variable,Units,Value
1,0,0,1,0,5.19000
2,1,0,2,0,0.08875
3,0,0,3,0,13.84500
4,1,0,1,0,1.76000
5,0,0,0,0,26.34600


,Flagged as Suspect Reading
1,0
2,0
3,0
4,0
5,0


,Sensor,Name,Variable,Units,Value
201,0,0,0,0,19.250
202,0,0,3,0,13.845
203,1,0,1,0,1.330
204,0,0,1,0,5.200
205,1,0,2,0,1.940


,Flagged as Suspect Reading
201,0
202,0
203,0
204,0
205,0


In [14]:
train['Flagged as Suspect Reading'].value_counts()
test['Flagged as Suspect Reading'].value_counts()

0    191
1      9
Name: Flagged as Suspect Reading, dtype: int64

0    63
1     3
Name: Flagged as Suspect Reading, dtype: int64

In [15]:
x_train.info()
y_train.info()
x_test.info()
y_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 1 to 200
Data columns (total 5 columns):
Sensor      200 non-null int64
Name        200 non-null int64
Variable    200 non-null int64
Units       200 non-null int64
Value       200 non-null float64
dtypes: float64(1), int64(4)
memory usage: 7.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 1 to 200
Data columns (total 1 columns):
Flagged as Suspect Reading    200 non-null int64
dtypes: int64(1)
memory usage: 1.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 201 to 266
Data columns (total 5 columns):
Sensor      66 non-null int64
Name        66 non-null int64
Variable    66 non-null int64
Units       66 non-null int64
Value       66 non-null float64
dtypes: float64(1), int64(4)
memory usage: 2.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 201 to 266
Data columns (total 1 columns):
Flagged as Suspect Reading    66 non-null int64
dtypes: int64(1)
memory usage: 660.0 b

In [16]:
x_train.shape
y_train.shape
x_test.shape
y_test.shape

(200, 5)

(200, 1)

(66, 5)

(66, 1)

In [17]:
type(x_train)
type(y_train)
type(x_test)
type(y_test)

pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

pandas.core.frame.DataFrame

In [18]:
x_train.values

array([[ 0.     ,  0.     ,  1.     ,  0.     ,  5.19   ],
       [ 1.     ,  0.     ,  2.     ,  0.     ,  0.08875],
       [ 0.     ,  0.     ,  3.     ,  0.     , 13.845  ],
       [ 1.     ,  0.     ,  1.     ,  0.     ,  1.76   ],
       [ 0.     ,  0.     ,  0.     ,  0.     , 26.346  ],
       [ 0.     ,  0.     ,  4.     ,  0.     , 10.727  ],
       [ 1.     ,  0.     ,  5.     ,  1.     ,  2.69   ],
       [ 0.     ,  0.     ,  6.     ,  2.     , 10.323  ],
       [ 0.     ,  0.     ,  7.     ,  0.     , 34.48484],
       [ 1.     ,  0.     ,  1.     ,  0.     ,  1.81   ],
       [ 2.     ,  0.     ,  2.     ,  0.     , -3.985  ],
       [ 0.     ,  0.     ,  2.     ,  0.     ,  7.5225 ],
       [ 1.     ,  0.     ,  3.     ,  0.     ,  3.859  ],
       [ 0.     ,  0.     ,  5.     ,  1.     ,  3.6    ],
       [ 0.     ,  0.     ,  2.     ,  0.     , 21.49625],
       [ 1.     ,  0.     ,  0.     ,  0.     , 13.319  ],
       [ 0.     ,  0.     ,  7.     ,  0.     , 32.48828

In [19]:
#confusion Matrix
def ConfusionMatrix(x,y):
    a = confusion_matrix(x,y)
    print(a)
    b = (a[0][0]+a[1][1])/(a[1][1]+a[0][1]+a[0][0]+a[1][0])
    return round(b*100,2)

In [20]:
#Decision Tree
decision = DecisionTreeClassifier(max_depth=5).fit(x_train, y_train)
decision_yhat = decision.predict(x_test)

print(decision.score(x_test,y_test))

decision_acc= ConfusionMatrix(decision_yhat,y_test)
print(decision_acc)

f1_score(decision_yhat,y_test) *100

print(precision_score(decision_yhat,y_test) *100)
print(recall_score(decision_yhat,y_test) *100)

1.0
[[63  0]
 [ 0  3]]
100.0


100.0

100.0
100.0


In [21]:
# dt=DecisionTreeClassifier()
# dt.fit(x_train,y_train)
# dt.predict(x_test)
# a=dt.predict_proba(x_test)
# list(a)

In [22]:
# tree.plot_tree(dt.fit(x_train,y_train))